## Imports and Device

In [1]:
! pip install transformers
# ! pip3 install wandb
# ! pip install rouge_score

import pandas as pd
import numpy as np
import torch
import os, gc
import re

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

from torch import cuda, nn, optim
# from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
# import rouge_score
# import wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.0 MB/s eta 0:00:00


In [2]:
manual_seed = 595
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Read the Cleaned Data

### Define the paths

In [3]:
# # run locally
# text_path = '../formatted_cases/'
# file = '../../annotated_data.xlsx'
# REGEX = r';+'
# sup_path = '../annotated_sup/'
# multi_path = text_path + 'multiple_files/'

In [4]:
# run on Google Colab
from google.colab import drive
drive.mount('/content/gdrive')
text_path = '/content/gdrive/My Drive/595/formatted_cases/'
file = '/content/gdrive/My Drive/595/annotated_data.xlsx'
REGEX = r';+'
sup_path = '/content/gdrive/My Drive/595/annotated_sup/'
multi_path = text_path + 'multiple_files/'

Mounted at /content/gdrive


In [5]:
# wandb.login()
# wandb.init(project="RTB_Cases", entity="qmygrace")

### Clean the Dataframe

In [6]:
df = pd.read_excel(file)
df['What is the file number of the case?'] = df['What is the file number of the case?'].str.replace(' and ', ';')
df['What is the file number of the case?'] = df['What is the file number of the case?'].str.replace(' ', ';')
df['What is the file number of the case?'] = df['What is the file number of the case?'].str.replace('/', ';')
df['What is the file number of the case?'] = df['What is the file number of the case?'].str.strip(';')
df['What is the file number of the case?'] = df['What is the file number of the case?'].apply(lambda x: re.sub(REGEX, ';', x))
df['What is the file number of the case?'] = df['What is the file number of the case?'].str.replace('File;number:;', '')
df['What is the file number of the case?'] = df['What is the file number of the case?'].str.replace('TET-89650-18;TET-89650-18', 'TET-89650-18;TEL-90138-18')
df = df.fillna('Not stated')
df = df.replace('Not applicable', 'Not stated')
df.rename(columns={
    'If yes to the previous question, did the decision state these conditions would make moving particularly burdensome?':
    'If any of the children had mental, medical or physical conditions, did the decision state these conditions would make moving particularly burdensome?',
    'If yes to the previous question, which of the following were applicable to the tenant?':
    'If the tenant had difficulty finding alternative housing for any reason, which of the following were applicable to the tenant?'    
}, inplace=True)

df = df.iloc[:, 2:-2]

print(df.shape)
df.head(6)

(702, 50)


,What is the file number of the case?,What was the date of the hearing? [mm/dd/yyyy],What was the date of the decision? [mm/dd/yyyy],Who was the member adjudicating the decision?,What was the location of the landlord tenant board?,Did the decision state the landlord was represented?,Did the decision state the landlord attended the hearing?,Did the decision state the tenant was represented?,Did the decision state the tenant attended the hearing?,Did the decision state the landlord was a not-for-profit landlord (e.g. Toronto Community Housing)?,...,"If the tenant did propose a payment plan, did the member accept the proposed payment plan?","If a payment plan was ordered, what was the length of the payment plan?","Did the decision mention the tenant’s difficulty finding alternative housing for any reason e.g.physical limitations, reliance on social assistance, etc.?","If the tenant had difficulty finding alternative housing for any reason, which of the following were applicable to the tenant?",Did the decision state the tenant was given prior notice for the eviction?,"If the tenant was given prior notice for the eviction, how much notice was given?",Did the decisions state postponement would result in the tenant accruing additional arrears?,Which other specific applications of the landlord or the tenant were mentioned?,Did the decision mention the validity of an N4 eviction notice?,Were there detail(s) in the decision not captured by this questionnaire that should be included?
0,CEL-87788-19,2019-10-16 00:00:00,2020-06-04 00:00:00,Sonia Anwar-Ali,Toronto,Yes,Not stated,No,Not stated,No,...,Not stated,12,No,Not stated,No,Not stated,No,L2: Application to End a Tenancy and Evict a T...,No,Tenant was a single mother with no support fro...
1,CEL-90549-19,2020-01-22 00:00:00,2020-01-10 00:00:00,Shelby Whittick,Mississauga,Yes,Yes,No,Yes,No,...,No,Not stated,No,Not stated,Yes,Not stated,Yes,No other specific applications were mentioned,No,Not stated
2,TEL-94478-18,2018-10-31 00:00:00,2018-11-21 00:00:00,Ruth Carey (Vice Chair),Toronto,Yes,Yes,No,Yes,No,...,Not stated,Not stated,No,Not stated,Yes,Not stated,No,N13: Notice to End your Tenancy Because the La...,No,Previous decision TEL-92736-18 < This decision...
3,TEL-94493-18,2018-10-31 00:00:00,2018-11-21 00:00:00,Ruth Carey (Vice Chair),Toronto,Yes,Yes,No,Yes,No,...,Yes,1,No,Not stated,Yes,Not stated,No,No other specific applications were mentioned,No,There were 7 previous application for non-paym...
4,CEL-72994-18,2018-03-07 00:00:00,2018-03-14 00:00:00,Avril Cardoso,Mississauga,Yes,No,Yes,No,No,...,No,Not stated,No,Not stated,Yes,Not stated,No,No other specific applications were mentioned,No,Third Application by Landlord in past 6 months...
5,CEL-73021-18,2018-06-15 00:00:00,2018-06-18 00:00:00,Avril Cardoso,Mississauga,Yes,No,No,No,No,...,Not stated,Not stated,No,Not stated,Yes,Not stated,No,L1: Application to Evict a Tenant for Non-paym...,No,Tenant did not show up because hearing took pl...


In [7]:
# df.columns   #`Timestamp` is not the time of the case

In [8]:
df_unique = df.drop_duplicates(subset=['What is the file number of the case?'])
df_unique = df_unique.reset_index(drop=True)

print(df_unique.shape)
df_unique.head(6)

(682, 50)


,What is the file number of the case?,What was the date of the hearing? [mm/dd/yyyy],What was the date of the decision? [mm/dd/yyyy],Who was the member adjudicating the decision?,What was the location of the landlord tenant board?,Did the decision state the landlord was represented?,Did the decision state the landlord attended the hearing?,Did the decision state the tenant was represented?,Did the decision state the tenant attended the hearing?,Did the decision state the landlord was a not-for-profit landlord (e.g. Toronto Community Housing)?,...,"If the tenant did propose a payment plan, did the member accept the proposed payment plan?","If a payment plan was ordered, what was the length of the payment plan?","Did the decision mention the tenant’s difficulty finding alternative housing for any reason e.g.physical limitations, reliance on social assistance, etc.?","If the tenant had difficulty finding alternative housing for any reason, which of the following were applicable to the tenant?",Did the decision state the tenant was given prior notice for the eviction?,"If the tenant was given prior notice for the eviction, how much notice was given?",Did the decisions state postponement would result in the tenant accruing additional arrears?,Which other specific applications of the landlord or the tenant were mentioned?,Did the decision mention the validity of an N4 eviction notice?,Were there detail(s) in the decision not captured by this questionnaire that should be included?
0,CEL-87788-19,2019-10-16 00:00:00,2020-06-04 00:00:00,Sonia Anwar-Ali,Toronto,Yes,Not stated,No,Not stated,No,...,Not stated,12,No,Not stated,No,Not stated,No,L2: Application to End a Tenancy and Evict a T...,No,Tenant was a single mother with no support fro...
1,CEL-90549-19,2020-01-22 00:00:00,2020-01-10 00:00:00,Shelby Whittick,Mississauga,Yes,Yes,No,Yes,No,...,No,Not stated,No,Not stated,Yes,Not stated,Yes,No other specific applications were mentioned,No,Not stated
2,TEL-94478-18,2018-10-31 00:00:00,2018-11-21 00:00:00,Ruth Carey (Vice Chair),Toronto,Yes,Yes,No,Yes,No,...,Not stated,Not stated,No,Not stated,Yes,Not stated,No,N13: Notice to End your Tenancy Because the La...,No,Previous decision TEL-92736-18 < This decision...
3,TEL-94493-18,2018-10-31 00:00:00,2018-11-21 00:00:00,Ruth Carey (Vice Chair),Toronto,Yes,Yes,No,Yes,No,...,Yes,1,No,Not stated,Yes,Not stated,No,No other specific applications were mentioned,No,There were 7 previous application for non-paym...
4,CEL-72994-18,2018-03-07 00:00:00,2018-03-14 00:00:00,Avril Cardoso,Mississauga,Yes,No,Yes,No,No,...,No,Not stated,No,Not stated,Yes,Not stated,No,No other specific applications were mentioned,No,Third Application by Landlord in past 6 months...
5,CEL-73021-18,2018-06-15 00:00:00,2018-06-18 00:00:00,Avril Cardoso,Mississauga,Yes,No,No,No,No,...,Not stated,Not stated,No,Not stated,Yes,Not stated,No,L1: Application to Evict a Tenant for Non-paym...,No,Tenant did not show up because hearing took pl...


In [9]:
info_lst = df_unique.columns[2:-2]

raw_file_text = []

for i in range(len(df_unique)):
    file_no = df_unique.iloc[i,0]
    if not os.path.isfile(text_path+file_no+'.txt'):
        print(f'{file_no} not found. Going to the supplement directory.')
        # passed_cases.append(file_no)
        if not os.path.isfile(sup_path+file_no+'.txt'):
            print(f'{file_no} not found. Going to the multiple directory.')
            with open (multi_path+file_no+'.txt') as t:
                # file_no_lst = file_no.split(';')
                # print(file_no_lst)
                raw_file_text.append(t.read())
        else:
            with open (sup_path+file_no+'.txt') as t:
                raw_file_text.append(t.read())
    else:
        with open (text_path+file_no+'.txt') as t:
            # cases_info[-1]['text'] = t.read()
            raw_file_text.append(t.read())
            # raw_file_name.append(file_no+'.txt')

TET-89650-18;TEL-90138-18 not found. Going to the supplement directory.
TNL-00793-18;TNL-01183-18 not found. Going to the supplement directory.
TNL-00793-18;TNL-01183-18 not found. Going to the multiple directory.
TNL-03299-18;TNT-00589-17 not found. Going to the supplement directory.
TNL-03299-18;TNT-00589-17 not found. Going to the multiple directory.
TNL-04435-18;TNL-03907-18 not found. Going to the supplement directory.
HOL-02144-17;HOT-02146-17 not found. Going to the supplement directory.
TEL-87475-18;TET-86819-17;TET-88355-18 not found. Going to the supplement directory.
TEL-87475-18;TET-86819-17;TET-88355-18 not found. Going to the multiple directory.
SWL-08112-17;SWL-08113-17 not found. Going to the supplement directory.
SWL-12547-18;SWL-12548-18 not found. Going to the supplement directory.
SWL-12547-18;SWL-12548-18 not found. Going to the multiple directory.
SWL-13901-18;SWT-14627-18 not found. Going to the supplement directory.
TEL-77442-17;TET-77790-17 not found. Going to 

In [10]:
# remove columns that have too little information
little_info_col = [15, 16, 26, 27, 28, 29, 30, 31, 41, 43, 45]
to_del = [df_unique.columns[i] for i in little_info_col]
for col in to_del:
    del df_unique[col]
to_del

['If any rent increases occurred, what was the rent after the increase(s)?',
 'If any rent increases occurred, when did the rent increase(s) come into effect? ',
 'How many total children did the tenant have living with them? ',
 'How many total children aged 17 or younger did the tenant have living with them?',
 'How many total children aged 13 or younger did the tenant have living with them? ',
 'How many total children aged 4 or younger did the tenant have living with them?',
 'Did the decision state any of the children had mental, medical or physical conditions?',
 'If any of the children had mental, medical or physical conditions, did the decision state these conditions would make moving particularly burdensome?',
 'If a payment plan was ordered, what was the length of the payment plan? ',
 'If the tenant had difficulty finding alternative housing for any reason, which of the following were applicable to the tenant?',
 'If the tenant was given prior notice for the eviction, how mu

In [11]:
del to_del
gc.collect()

1103

### Split the Train Dataframe and Validation Dataframe

In [12]:
train_df = df_unique.iloc[:620, :]
val_df = df_unique.iloc[620:, :].reset_index(drop=True)
train_df.shape, val_df.shape

((620, 39), (62, 39))

In [13]:
for i, q in enumerate(train_df.columns):
    print(i, q)

0 What is the file number of the case?
1 What was the date of the hearing? [mm/dd/yyyy]
2 What was the date of the decision? [mm/dd/yyyy]
3 Who was the member adjudicating the decision?
4 What was the location of the landlord tenant board?
5 Did the decision state the landlord was represented?
6 Did the decision state the landlord attended the hearing?
7 Did the decision state the tenant was represented?
8 Did the decision state the tenant attended the hearing?
9 Did the decision state the landlord was a not-for-profit landlord (e.g. Toronto Community Housing)?
10 Did the decision state the tenant was collecting a subsidy?
11 What was the outcome of the case?
12 What was the length of the tenancy, or in other words, how long had the tenants lived at the residence in question? 
13 What was the monthly rent?
14 What was the amount of the rental deposit? 
15 What was the total amount of arrears?
16 Over how many months did the arrears accumulate? 
17 If the tenant made a payment on the ar

## Initialize the Tokenizer and the Model

In [14]:
# @article{guo2021longt5,
#   title={LongT5: Efficient Text-To-Text Transformer for Long Sequences},
#   author={Guo, Mandy and Ainslie, Joshua and Uthus, David and Ontanon, Santiago and Ni, Jianmo and Sung, Yun-Hsuan and Yang, Yinfei},
#   journal={arXiv preprint arXiv:2112.07916},
#   year={2021}
# }

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-local-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/long-t5-local-base")

## A Test before Finetuning

In [15]:
def prompt(dataframe, raw_texts):
    input_texts = []
    outputs = []
    # long_cases = 0
    
    questions = dataframe.columns
    
    for q_no in range(len(questions)):
        answers = dataframe.iloc[:,q_no]
        # print(len(raw_texts), len(answers))
        assert len(raw_texts) == len(answers)

        for i in range(len(answers)):
            full_text = raw_texts[i]
            text = full_text[full_text.find('Content:')+len('Content:'):]

            # if len(text) > 26000:
            #     # print(len(text))
            #     text = text[:26000]
            #     long_cases += 1

            text = text.replace('\n', ' ')
            text = text.replace('\xa0', ' ')
            text = text.replace('\t', ' ')
            text = text.replace('   ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ')
            # text
            # for word in stop_words:
            #     text = text.replace(' '+word+' ', ' ')

            if 'Schedule 1' in text:
                s_idx = text.find('Schedule 1')
                text = text[:s_idx]

            input_text = f'Question: {questions[q_no]} Text: {text}'  
            input_texts.append(input_text)

            output = str(answers[i])
            outputs.append(output)
        
    # print(len(input_texts), len(outputs))
    # print(input_texts[0], outputs[0]) 
    # print(long_cases)
    return input_texts, outputs

In [16]:
q1_lst, a1_lst = prompt(df_unique, raw_file_text)
q1 = q1_lst[0]
a1 = a1_lst[0]
# # print(q1)
input_encoding = tokenizer(q1)
output_encoding = tokenizer(a1)
input_ids = torch.LongTensor(input_encoding['input_ids']).unsqueeze(0)  # batch of size 1
attention_mask = torch.LongTensor(input_encoding['attention_mask']).unsqueeze(0)
# attention_mask[:, [1, 4, 21,]] =  2  # Set global attention based on the task. For example,
                                     # classification: the <s> token
                                     # QA: question tokens
print(input_ids.shape, attention_mask.shape)
# input_ids, attention_mask = pad_to_window_size(
#         input_ids, attention_mask, config.attention_window[0], tokenizer.pad_token_id)

output = model.generate(
    input_ids=input_ids, 
    attention_mask=attention_mask,
    return_dict_in_generate=True, 
    output_scores=False, 
    max_length=512,
    temperature=0.5,
    do_sample=True,
    repetition_penalty=3.0,
    top_k=10)


torch.Size([1, 1399]) torch.Size([1, 1399])


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [17]:
tokenizer.batch_decode(output['sequences'])

['<pad>),,).”!)?)”,],The The Her ThereThere It OnOn ONonset augment annotation")”) her His." awakening mindfulness;);",,” FromFromfrom FROM voyage create creatingCreating invented Their their your Your hisgekommen gegründetgetroffen gewonnennimmt victory fêteHôtel beside upon onUpon began begins begin commenceize workplace desire They theyoire In inING doing did Doplaîtmenéerisen Rising emergedévidence THEthe But quarterback pianist windshield apprenticeshipentrepreneurship Around about gesture," him Him monnaie replenish reconnect verbinden beinhaltet erreichtgefahren geworden gewesen begleiten Bildung Führung Forschung zur toward AfterAfter:?”!” studied discussed discussing Youyouanthropo crank1,2,Founded soothe plante Marseille humaine réalisé réalisée élabor oppos contrary invited enthusiasmigueur amidmidst aftermath coincidenceistoriaemiaificareêtre animé Gegnerräumen mathematics Mathematics played playing play plays sketches onlineateaise(2005) (2004) 2017,6, // disponibleforemen

## Preprocess the Data

In [18]:
def preprocess(dataframe, tokenizer, raw_texts):
    input_texts, outputs = prompt(dataframe, raw_texts)   
    
    input_toks = tokenizer.batch_encode_plus(input_texts,
                                             add_special_tokens=False, 
                                             return_token_type_ids=False)
    output_toks = tokenizer.batch_encode_plus(outputs, 
                                              add_special_tokens=False,
                                              return_token_type_ids=False)
    # print(len(q1_train_input['input_ids']), len(q1_train_output['input_ids']))
    return input_toks, output_toks
    

In [19]:
train_raw_texts = raw_file_text[:620]
val_raw_texts = raw_file_text[620:]

In [20]:
train_input, train_output = preprocess(train_df, tokenizer, train_raw_texts)
val_input, val_output = preprocess(val_df, tokenizer, val_raw_texts)

In [21]:
# len(q1_train_input['input_ids']), len(q1_train_output['input_ids'])

In [22]:
print("Input length:", len(train_input))
# print(" ")
print("Input example:\n", tokenizer.decode(train_input['input_ids'][0])[:100])
print(" ")
print("Input ID example:\n", train_input['input_ids'][0][:100])
print(" ")
print("Tokens:\n", [tokenizer.convert_ids_to_tokens(id) for id in train_input['input_ids'][0]][:100])
print(" ")
print("Attention Mask:", train_input['attention_mask'][0])
print(" ")
print("Output example:\n", tokenizer.decode(train_output['input_ids'][0])[:100])
print(" ")
print("Output ID example:\n", train_output['input_ids'][0])
print(" ")

Input length: 2
Input example:
 Question: What is the file number of the case? Text: Order under Section 69 Residential Tenancies Ac
 
Input ID example:
 [11860, 10, 363, 19, 8, 1042, 381, 13, 8, 495, 58, 5027, 10, 5197, 365, 5568, 3, 3951, 15773, 4738, 28704, 1983, 6, 3581, 7344, 7720, 10, 205, 3577, 18, 27697, 4060, 4481, 283, 5, 254, 5, 41, 532, 3, 31, 434, 232, 322, 26, 31, 61, 2930, 21, 46, 455, 12, 13813, 8, 3, 324, 6833, 11, 3, 15, 7287, 17, 283, 5, 439, 5, 41, 532, 3, 31, 382, 35, 288, 31, 61, 250, 3, 88, 65, 118, 15803, 120, 1480, 16, 3788, 112, 3170, 5, 37, 2216, 322, 26, 92, 7760, 6107, 21, 284, 239, 8, 4738]
 
Tokens:
 ['▁Question', ':', '▁What', '▁is', '▁the', '▁file', '▁number', '▁of', '▁the', '▁case', '?', '▁Text', ':', '▁Order', '▁under', '▁Section', '▁', '69', '▁Residential', '▁Ten', 'ancies', '▁Act', ',', '▁2006', '▁File', '▁Number', ':', '▁C', 'EL', '-', '877', '88', '-19', '▁M', '.', 'C', '.', '▁(', 'the', '▁', "'", 'L', 'and', 'lor', 'd', "'", ')', '▁applied', '▁fo

## Create the Dataset

In [23]:
PAD = tokenizer.pad_token_id
SEP = tokenizer.sep_token_id
PAD, SEP

(0, None)

In [24]:
class CaseDataset(Dataset):

    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        input_ids = self.inputs['input_ids'][idx]
        attention_mask = self.inputs['attention_mask'][idx]

        target_ids = self.outputs['input_ids'][idx]
        # target_attention_mask = self.outputs['attention_mask'][idx]
        return {"input_ids": input_ids, "attention_mask":attention_mask, "output_ids":target_ids}


def collate_fn(batch):
    batch_input = [torch.LongTensor(example['input_ids']) for example in batch]
    batch_output = [torch.LongTensor(example['output_ids']) for example in batch]
    batch_mask = [torch.LongTensor(example['attention_mask']) for example in batch]

    padded_batch_input_ids = pad_sequence(batch_input, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_batch_label = pad_sequence(batch_output, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_batch_att_mask = pad_sequence(batch_mask, batch_first=True, padding_value=-100)

    return {"input_ids": padded_batch_input_ids, "attention_mask": padded_batch_att_mask, "labels": padded_batch_label}


def to_device(data, device):
    new_data = {}
    for k in data:
        new_data[k] = data[k].to(device)
    return new_data

## Prepare the Functions for Training and Evaluation

In [25]:
def train(model:nn.Module, train_loader:DataLoader, optimizer:optim.Optimizer, log_step=200):
    model.train()
    epoch_loss = 0.0
    log_loss = 0.0
    for idx, batch in enumerate(train_loader):
        # try:
        model.zero_grad()
        batch = to_device(batch, device)
        loss = model(**batch).loss
        # print(loss)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        log_loss += loss.item()

        # wandb.log({'batch':idx, 'train_loss': loss.item()})
        # wandb.log({'batch':idx, 'accumulated_train_loss_in_this_Q': log_loss})

        if idx % log_step == 0:
            print(f"Train Step: {idx} Loss: {log_loss / log_step}")
            log_loss = 0.0
        # except:
        #     print(f'The text is too long. Passing for now. Step No: {idx}')
        #     pass

    return epoch_loss / len(train_loader)
        

@torch.no_grad()
def evaluate(model:nn.Module, eval_loader:DataLoader):
    eval_loss = 0.0
    correct = 0
    total = 0
    model.eval()
    for batch in eval_loader:
        batch = to_device(batch, device)
        output = model(**batch)
        loss = output.loss
        eval_loss += loss.item()
        pred = output.logits.argmax(-1)
        label = batch["labels"]
        correct += torch.where(label!=-100, pred==label, 0).sum().item()
        total += torch.sum(label != -100).item()
    
    print(total, correct)

    eval_acc = correct / total
    eval_loss = eval_loss / len(eval_loader) 
    return eval_acc, eval_loss


In [26]:
@torch.no_grad()
def answer(model, loader):
    all_preds = []
    all_labels = []
    model.eval()
    for batch in loader:
        batch = to_device(batch, device)
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        outputs = model.generate(input_ids=input_ids, 
                                 attention_mask=attention_mask, 
                                 return_dict_in_generate=True, 
                                 pad_token_id=tokenizer.pad_token_id, 
                                 max_length=512, 
                                 top_k=15)
        
        decode_texts = tokenizer.batch_decode([l[l != 0] for l in outputs['sequences']])
        gold_texts = tokenizer.batch_decode([l[l != 0] for l in labels])
        # print(decode_texts, gold_texts)
        for gold, decode in zip(gold_texts, decode_texts):
            l = gold.replace('</s>', '').replace('<pad>','').replace('<s>', '')
            p = decode.replace('</s>', '').replace('<pad>','').replace('<s>', '')

            # if '<pad>' in gold:
            #     l_pad_idx = gold.index('<pad>')
            #     l = gold[:l_pad_idx].replace(' ', '').replace('</s>', '').replace('<pad>','').replace('<s>', '')
            # else:
            #     l = gold.replace(' ', '').replace('</s>', '').replace('<pad>','').replace('<s>', '')
            
            # if '<pad>' in decode:
            #     p_pad_idx = decode.index('<pad>')
            #     p = decode[:p_pad_idx].replace(' ', '').replace('</s>', '').replace('<pad>','').replace('<s>', '')
            # else:
            #     p = decode.replace(' ', '').replace('</s>', '').replace('<pad>','').replace('<s>', '')

            # print(l, p)
            all_labels.append(l)
            all_preds.append(p)
    
    return all_preds, all_labels


def accuracy(sys, gold):
    total = 0
    correct = 0
    for s, g in zip(sys, gold):
        if s == g:
            correct += 1
        total += 1
            
    accuracy = correct / total
    return accuracy, correct, total


## Train the Model

In [27]:
model.to(device)

LongT5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): LongT5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): LongT5Block(
        (layer): ModuleList(
          (0): LongT5LayerLocalSelfAttention(
            (LocalSelfAttention): LongT5LocalAttention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): LongT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): LongT5LayerFF(
            (DenseReluDense): LongT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out

In [28]:
# Experiment
train_dataset = CaseDataset(train_input, train_output)
train_loader = DataLoader(train_dataset, batch_size=2, collate_fn=collate_fn, shuffle=True)

val_dataset = CaseDataset(val_input, val_output) 
val_loader = DataLoader(val_dataset, batch_size=2, collate_fn=collate_fn, shuffle=False)


In [29]:
del df, train_dataset
gc.collect()

12

In [30]:
# experiment
epochs = 3
optimizer = optim.Adam(model.parameters(), lr=5e-5)

model.train()

# print(f"Training Question 1")
for epoch in range(epochs):
    print(f"Epoch {epoch+1}:")
    
    train_loss = train(model, train_loader, optimizer)
    print(f"Epoch {epoch+1} Training Loss: {train_loss}")

    eval_acc, eval_loss = evaluate(model, val_loader)
    print(f"Epoch {epoch} Eval Acc: {eval_acc}; Eval Loss: {eval_loss}")


Epoch 1:


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Train Step: 0 Loss: 1.822818603515625
Train Step: 200 Loss: 123.64621903061867
Train Step: 400 Loss: 38.84924390259192
Train Step: 600 Loss: 28.70413179580908
Train Step: 800 Loss: 24.401120125410962
Train Step: 1000 Loss: 18.299381642640302
Train Step: 1200 Loss: 17.27281296124327
Train Step: 1400 Loss: 16.185186962220033
Train Step: 1600 Loss: 14.671331882543855
Train Step: 1800 Loss: 15.00521946764551
Train Step: 2000 Loss: 13.150630525608937
Train Step: 2200 Loss: 12.603646983891377
Train Step: 2400 Loss: 12.710760195199065
Train Step: 2600 Loss: 11.562217021898618
Train Step: 2800 Loss: 10.766249047890305
Train Step: 3000 Loss: 11.288789272617892
Train Step: 3200 Loss: 11.472912987380987
Train Step: 3400 Loss: 10.51098533523269
Train Step: 3600 Loss: 10.134514307946228
Train Step: 3800 Loss: 9.057413031242358
Train Step: 4000 Loss: 9.241682777403621
Train Step: 4200 Loss: 9.435392955848947
Train Step: 4400 Loss: 7.972075472590514
Train Step: 4600 Loss: 8.747431805165252
Train Step

In [31]:
torch.save(model.state_dict(), '/content/gdrive/My Drive/595/longT5_2epoch_law_allqs.pt')

In [32]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 247,577,856 trainable parameters


## Evaluate the Model on Validation Set

In [52]:
def q_prompt(dataframe, q_no, raw_texts):
    input_texts = []
    outputs = []
    # long_cases = 0
    
    questions = dataframe.columns
    answers = dataframe.iloc[:,q_no]
    # print(len(raw_texts), len(answers))
    assert len(raw_texts) == len(answers)

    for i in range(len(answers)):
        full_text = raw_texts[i]
        text = full_text[full_text.find('Content:')+len('Content:'):]

        # if len(text) > 26000:
        #     # print(len(text))
        #     text = text[:26000]
        #     long_cases += 1

        text = text.replace('\n', ' ')
        text = text.replace('\xa0', ' ')
        text = text.replace('\t', ' ')
        text = text.replace('   ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ')
        # text
        # for word in stop_words:
        #     text = text.replace(' '+word+' ', ' ')

        if 'Schedule 1' in text:
            s_idx = text.find('Schedule 1')
            text = text[:s_idx]

        input_text = f'Question: {questions[q_no]} Text: {text}'  
        input_texts.append(input_text)

        output = str(answers[i])
        outputs.append(output)
        
    # print(len(input_texts), len(outputs))
    # print(input_texts[0], outputs[0]) 
    # print(long_cases)
    return input_texts, outputs

In [53]:
def q_preprocess(dataframe, q_no, tokenizer, raw_texts):
    input_texts, outputs = q_prompt(dataframe, q_no, raw_texts)   
    
    input_toks = tokenizer.batch_encode_plus(input_texts,
                                             add_special_tokens=False, 
                                             return_token_type_ids=False)
    output_toks = tokenizer.batch_encode_plus(outputs, 
                                              add_special_tokens=False,
                                              return_token_type_ids=False)
    # print(len(q1_train_input['input_ids']), len(q1_train_output['input_ids']))
    return input_toks, output_toks
    

In [54]:
def get_test_dataloader(df, q_no, tokenizer, raw_texts):
    input_toks, output_toks = q_preprocess(df, q_no, tokenizer, raw_texts)
    dataset = CaseDataset(input_toks, output_toks)
    dataloader = DataLoader(dataset, 
                            batch_size=16, 
                            collate_fn=collate_fn, 
                            shuffle=False)
    return dataloader

In [55]:
def answer_qs(val_df, q_no, tokenizer):
    loader = get_test_dataloader(val_df, q_no, tokenizer, val_raw_texts)
    # print(len(loader))
    
    questions = val_df.columns
    print(f'Q{q_no+1}: {questions[q_no]}')
    
    preds, golds = answer(model, loader)
    acc, correct, total = accuracy(preds, golds)
    acc = round(acc, 5)
    
    print(f"Accuracy for this question is: {acc*100}%")
    print('')
    
    return acc, preds

In [48]:
# del train_df, train_loader, count_parameters
# gc.collect()

In [60]:
acc_lst = []
with open ('/content/gdrive/My Drive/595/longT5_allqs_preds.txt', 'w') as p:
    for i in range(0, val_df.shape[1]):
        p.write(f'Q{i+1}: {val_df.columns[i]}')
        acc, preds = answer_qs(val_df, i, tokenizer)
        acc_lst.append(acc)
        print(preds)
        print(' ')
        p.write(str(preds)+'\n')
        p.write('\n')
avg_acc = sum(acc_lst) / len(acc_lst)

Q1: What is the file number of the case?
Accuracy for this question is: 0.0%

['TEL-88989-18', 'TEL-97979-17', 'TEL-98989-18', 'TEL-98989-17', 'TEL-98989-18', 'TEL-97979-17-RV-RV-0245-17-RV-02-02 00:00:00', 'TEL-98989-18', 'TEL-98989-18', 'TEL-97979-17', 'TEL-97979-17', 'TEL-98989-18', 'TEL-97979-17', 'TEL-98989-18', 'TEL-97979-17', 'TEL-98969-18', 'TEL-98989-17', 'TNL-070707-18', 'TEL-97979-17', 'TEL-98969-18', 'TEL-97979-17', 'TEL-98989-18-RV-RV-RV-0645-17-RV-RV-02-06 00:00:00', 'TEL-97979-17', 'TEL-98969-18', 'TEL-98989-18', 'TNL-070707-18', 'TEL-98989-18', 'TEL-97979-17-RV-0245-17-RV-02-02 00:00:00', 'TEL-98989-18', 'TEL-98969-18', 'TEL-97979-17', 'TEL-98989-18', 'TEL-97979-17', 'TEL-98989-17', 'TEL-98989-18', 'TEL-97979-17', 'TEL-97979-17', 'TEL-98989-18', 'TEL-88969-18', 'TEL-98969-18', 'TEL-98989-18', 'TNL-0769-19', 'TEL-98989-18', 'TNL-070707-19', 'TEL-87979-17', 'TEL-97979-17-RV-RV-0245-17-RV-RV-0280-17', 'TEL-97976-17-RV-RV-0645-17-RV-RV-0280', 'TEL-97976-17', 'TEL-98989-18',

UnicodeEncodeError: ignored

In [65]:
del preds
gc.collect()

21629

In [66]:
# continue
with open ('/content/gdrive/My Drive/595/longT5_allqs_preds.txt', 'a', encoding='utf-8') as p:
    for i in range(28, val_df.shape[1]):
        p.write(f'Q{i+1}: {val_df.columns[i]}')
        acc, preds = answer_qs(val_df, i, tokenizer)
        acc_lst.append(acc)
        print(preds)
        print(' ')
        p.write(str(preds)+'\n')
        p.write('\n')
avg_acc = sum(acc_lst) / len(acc_lst)

Q29: What was the total income of the tenant’s household? 


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Accuracy for this question is: 91.935%

['Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated', 'Not stated']
 
Q30: Did the decision mention the tenant lost their job leading up to or during the perio

In [67]:
assert len(acc_lst) == 39
acc_lst

[0.0,
 0.0,
 0.0,
 0.04839,
 0.54839,
 0.69355,
 0.56452,
 0.62903,
 0.64516,
 0.98387,
 0.91935,
 0.56452,
 0.82258,
 0.59677,
 0.29032,
 0.03226,
 0.09677,
 0.69355,
 0.91935,
 0.8871,
 0.87097,
 0.98387,
 0.83871,
 0.67742,
 0.22581,
 0.87097,
 0.72581,
 0.90323,
 0.91935,
 0.95161,
 0.82258,
 0.72581,
 0.77419,
 0.85484,
 0.93548,
 0.43548,
 0.37097,
 0.8871,
 0.56452]

In [68]:
q1_loader = get_test_dataloader(val_df, 0, tokenizer, val_raw_texts)
# print(len(loader))

questions = val_df.columns
print(f'Q1: {questions[0]}')

preds, golds = answer(model, q1_loader)
preds, golds

Q1: What is the file number of the case?


(['TEL-88989-18',
  'TEL-97979-17',
  'TEL-98989-18',
  'TEL-98989-17',
  'TEL-98989-18',
  'TEL-97979-17-RV-RV-0245-17-RV-02-02 00:00:00',
  'TEL-98989-18',
  'TEL-98989-18',
  'TEL-97979-17',
  'TEL-97979-17',
  'TEL-98989-18',
  'TEL-97979-17',
  'TEL-98989-18',
  'TEL-97979-17',
  'TEL-98969-18',
  'TEL-98989-17',
  'TNL-070707-18',
  'TEL-97979-17',
  'TEL-98969-18',
  'TEL-97979-17',
  'TEL-98989-18-RV-RV-RV-0645-17-RV-RV-02-06 00:00:00',
  'TEL-97979-17',
  'TEL-98969-18',
  'TEL-98989-18',
  'TNL-070707-18',
  'TEL-98989-18',
  'TEL-97979-17-RV-0245-17-RV-02-02 00:00:00',
  'TEL-98989-18',
  'TEL-98969-18',
  'TEL-97979-17',
  'TEL-98989-18',
  'TEL-97979-17',
  'TEL-98989-17',
  'TEL-98989-18',
  'TEL-97979-17',
  'TEL-97979-17',
  'TEL-98989-18',
  'TEL-88969-18',
  'TEL-98969-18',
  'TEL-98989-18',
  'TNL-0769-19',
  'TEL-98989-18',
  'TNL-070707-19',
  'TEL-87979-17',
  'TEL-97979-17-RV-RV-0245-17-RV-RV-0280-17',
  'TEL-97976-17-RV-RV-0645-17-RV-RV-0280',
  'TEL-97976-17',
